In [46]:
import sys,os

DEMO_PATH_CURVATURE = 'demo-meshes/demo-meshes/curvatures'
DEMO_PATH_SMOOTHING = 'demo-meshes/demo-meshes/smoothing'
DEMO_PATH_DECOMPOSITION = 'demo-meshes/demo-meshes/decomposition'
EXAMPLE_NONTEXTURE_PATH = 'example_meshes_nontexture/example_meshes'
LIB_PATH = '../python_lib'

if not os.path.exists(DEMO_PATH_CURVATURE):
    print( 'cannot find \\DEMO_PATH_CURVATURE, please update DEMO_PATH_CURVATURE')
    exit(1)
else:
    print('found resources')
    
if not os.path.exists(DEMO_PATH_SMOOTHING):
    print( 'cannot find \\DEMO_PATH_SMOOTHING, please update DEMO_PATH_SMOOTHING')
    exit(1)
else:
    print('found resources')

if not os.path.exists(DEMO_PATH_SMOOTHING):
    print( 'cannot find \\DEMO_PATH_SMOOTHING, please update DEMO_PATH_SMOOTHING')
    exit(1)
else:
    print('found resources')
    
if not os.path.exists(EXAMPLE_NONTEXTURE_PATH):
    print( 'cannot find \\EXAMPLE_NONTEXTURE, please update DEMO_PATH')
    exit(1)
else:
    print('found more meshes')

# append path 
sys.path.append(LIB_PATH) 
from geo_tools import rd_helper

import pyglet
pyglet.options['shadow_window'] = False

import pyrender
import numpy as np
import trimesh

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.spatial import KDTree
from scipy.spatial import cKDTree
import math
from math import cos, sin
from scipy.spatial.transform import Rotation
from sympy import *
from scipy.sparse.linalg import eigs, eigsh, cg

found resources
found resources
found resources
found more meshes


### Some handy functions for display

In [3]:
def scene_factory(render_list):
    
    scene = pyrender.Scene(ambient_light=0.5*np.array([1.0, 1.0, 1.0, 1.0]))
    
    for m in render_list:
        scene.add(m)
    
    return scene

def run_gui(render_list, **kargs ):
    scene = scene_factory(render_list)
    # call GUI
    v=pyrender.Viewer(scene, use_raymond_lighting=True, **kargs)
    # free resources
    del v 
    return None

def add_colour_to_mesh(mesh, colour_pallete):
    mesh.visual.vertex_colors = np.tile(colour_pallete, (mesh.vertices.shape[0],1))
    return mesh

def normal_shading(mesh): 
    mesh.visual.vertex_colors = mesh.vertex_normals * 255
    return mesh

def curvature_shading(mesh, colours): 
    mesh.visual.vertex_colors = colours
    return mesh

## Get laplacian operator

### Find the one ring neigbours of a vertice

#### Import mesh

In [6]:
# m1_fp = os.path.join(DEMO_PATH_CURVATURE,'lilium.obj')
m1_fp = os.path.join('bumpy-cube-small.obj')
m2_fp = os.path.join(DEMO_PATH_CURVATURE,'plane.obj')
m3_fp = os.path.join(DEMO_PATH_CURVATURE,'lilium.obj')


assert os.path.exists(m1_fp), 'cannot found:'+m1_fp 
assert os.path.exists(m2_fp), 'cannot found:'+m2_fp 
assert os.path.exists(m3_fp), 'cannot found:'+m3-fp 

M1 = trimesh.load(m1_fp) 
M2 = trimesh.load(m2_fp) 
M3 = trimesh.load(m3_fp)

## access mesh properties

In [7]:
props= ['vertices', 'faces', 'edges', 'face_normals' , 'vertex_normals', 'vertex_neighbors']

for p in props:
    print('M1 {:}, shape={:}, #={:}'.format(p,str(getattr(M1,p).shape),getattr(M1,p).shape[0]))
    

M1 vertices, shape=(1502, 3), #=1502
M1 faces, shape=(3000, 3), #=3000
M1 edges, shape=(9000, 2), #=9000
M1 face_normals, shape=(3000, 3), #=3000
M1 vertex_normals, shape=(1502, 3), #=1502
M1 vertex_neighbors, shape=(1502,), #=1502


## Get laplace beltrami

In [4]:
def get_uniform_discretization(mesh):
    vertice_number = len(mesh.vertices)
    laplace_beltrami = 1 * np.eye(vertice_number)
    for i in range(vertice_number):
        neigbours =  mesh.vertex_neighbors[i]
        valence = len(neigbours)
        laplace_beltrami[i, neigbours] = -1/valence
    return laplace_beltrami

## Get $\triangle{x}$ 

In [5]:
def get_del_x(mesh):
    laplacian = get_uniform_discretization(mesh)
    del_x = laplacian @ mesh.vertices
    return del_x

In [13]:
del_x = get_del_x(M1)

### Get mean curvature

In [6]:
def get_mean_curvature(del_x, mesh):
    H_abs = np.linalg.norm(del_x, axis=1)/2
    H_n = del_x/-2
#     print(H_n)
    n = mesh.vertex_normals
    dot_prod_Hn_n = np.sum(H_n*n, axis=1)
    H_sign = np.sign(dot_prod_Hn_n)
    H = H_sign * H_abs
    return H

In [14]:
H = get_mean_curvature(del_x, M1)

[[ 0.00648417  0.078867    0.02683483]
 [-0.01050521 -0.04780564 -0.01618921]
 [-0.0384172   0.0113338   0.0880604 ]
 ...
 [-0.01970643 -0.00592264 -0.04967143]
 [-0.0017537  -0.0352829   0.0082849 ]
 [ 0.0017632   0.0012126   0.0042816 ]]


## visualize mean curvature

In [7]:
def get_vertice_colours_from_list(list_values , cmap=cm.hot):
    minima = min(list_values)
#     print(minima)
    
    maxima = max(list_values)
#     print(maxima)
    
    norm = matplotlib.colors.Normalize(vmin=minima, vmax=maxima)
    mapper = cm.ScalarMappable(norm=norm, cmap=cmap)
    
    colours =  mapper.to_rgba(list_values)[:,[0,1,2]] * 255
    return colours

In [16]:
colours_mean_curvature =  get_vertice_colours_from_list(H,cm.hot)

-0.2503869328693073
0.15603051317564193


In [17]:
curvature_shading(M1,colours_mean_curvature)
m1_mesh_rd = pyrender.Mesh.from_trimesh(M1)
run_gui([m1_mesh_rd])

In [344]:
# angle_defect = M1.vertex_defects[1000]
# print(angle_defect)
        
# #vertex_faces : faces which a vertex belongs to -1 for padding,remove them
# faces_vertex_belongs_to = M1.vertex_faces[1000][M1.vertex_faces[1000] != -1]
# print(faces_vertex_belongs_to)

# #Area of faces 
# one_ring_area = np.sum(M1.area_faces[faces_vertex_belongs_to])
# print(one_ring_area)

# barycentric_area = one_ring_area/3

# curv_gauss = angle_defect/barycentric_area
# print(curv_gauss)

## Gauss Curvature

In [18]:
def get_gauss_curvature (mesh):
    #amount of vertices
    vertice_number = len(mesh.vertices)
    
    #for each vertex,
    gauss_curvature = np.zeros(vertice_number)
    for i in range(vertice_number):
        # vertex_defects : 2pi - sum of angles of a vertex
        angle_defect = mesh.vertex_defects[i]
        
        #vertex_faces : faces which a vertex belongs to -1 for padding,remove them
        faces_vertex_belongs_to = mesh.vertex_faces[i][mesh.vertex_faces[i] != -1]
        
        #Area of faces 
        one_ring_area = np.sum(mesh.area_faces[faces_vertex_belongs_to])
        
        barycentric_area = one_ring_area/3
        
        gauss_curvature[i] = angle_defect/barycentric_area
    return gauss_curvature    

In [19]:
M1_gauss_curvature = get_gauss_curvature(M1)

## visualize gauss curvature

In [20]:
colours_gauss_curvature =  get_vertice_colours_from_list(M1_gauss_curvature)

-1.6527433074218163
2.7309242726060217


In [21]:
curvature_shading(M1,colours_gauss_curvature)
m1_mesh_rd = pyrender.Mesh.from_trimesh(M1)
run_gui([m1_mesh_rd])

## Question 2: cotan

In [8]:
def get_barycentric_area(mesh, vertex):
    
    #vertex_faces : faces which a vertex belongs to -1 for padding,remove them
    faces_vertex_belongs_to = mesh.vertex_faces[vertex][mesh.vertex_faces[vertex] != -1]

    #Area of faces 
    one_ring_area = np.sum(mesh.area_faces[faces_vertex_belongs_to])
#     print("one_ring_area", one_ring_area )

    barycentric_area = one_ring_area/3
#     print("barycentric_area", barycentric_area)
    
    return barycentric_area

In [9]:
def get_alpha_beta(mesh, vertex, neigbour_vertex):
    faces_vertex_belongs_to = mesh.vertex_faces[vertex][mesh.vertex_faces[vertex] != -1]
    faces_neighbour_belongs_to = mesh.vertex_faces[neigbour_vertex][mesh.vertex_faces[neigbour_vertex] != -1]
#     print("get_alpha_beta.................................")
#     print("faces_vertex_belongs_to",faces_vertex_belongs_to)
#     print("faces_neighbour_belongs_to", faces_neighbour_belongs_to)
    
    shared_faces =  faces_vertex_belongs_to[np.isin(faces_vertex_belongs_to,faces_neighbour_belongs_to )]  #2 values
#     print("shared_faces", shared_faces)
    
    alpha = 0
    beta = 0
    if (len(shared_faces) == 1):
        face_vertices = mesh.faces[shared_faces]
#         print("face_vertices", face_vertices)
        angles_of_face =  mesh.face_angles[shared_faces]
#         print("angles_of_face",angles_of_face)
        alpha_mask =  np.isin(face_vertices,[vertex,neigbour_vertex])
#         print("alpha_mask", alpha_mask)
        alpha = angles_of_face[~alpha_mask][0]
#         print("alpha")
        
    else:
        angles_of_shared_faces =  mesh.face_angles[shared_faces] #2x3 size
#         print("angles_of_shared_faces",angles_of_shared_faces)

        shared_faces_vertices = mesh.faces[shared_faces] #2x3 size
#         print("shared_faces_vertices",shared_faces_vertices)

        alpha = angles_of_shared_faces[0][~np.isin(shared_faces_vertices[0],shared_faces_vertices[1])][0]
        beta = angles_of_shared_faces[1][~np.isin(shared_faces_vertices[1],shared_faces_vertices[0])][0] 
        
    
    return [alpha,beta]

In [12]:
def get_cot_sum_vertex_difference(mesh, vertex, alpha_beta, neighbor_index):
#     print("alpha_beta", alpha_beta)
#     print("get_cot_sum_vertex_difference....................................")
    cot_val = 0
    if alpha_beta[1] == 0:
        cot_val = cot(alpha_beta[0])
#         print("in 1")
    else:
#         print("in 2")
        cot_val = cot(alpha_beta[0]) + cot(alpha_beta[1])
    
    if cot_val == zoo:
        cot_val = 0
#     print("cot_val", cot_val)
    
    #fvi-fv
    vertex_difference = mesh.vertices[neighbor_index] - mesh.vertices[vertex]
#     print("vertex_difference", vertex_difference)
    cot_sum_vertex_difference = cot_val * vertex_difference
#     print("cot_sum_vertex_difference", cot_sum_vertex_difference)
    return cot_sum_vertex_difference
    

In [13]:
def get_cotan_discretization(mesh):
#     print("get_cotan_discretization..........................................")
    vertice_number = len(mesh.vertices)
    cotan_discretization = np.zeros([vertice_number,3])
    for vertex in range(vertice_number):
        one_ring_neighbours = mesh.vertex_neighbors[vertex]
#         print("one_ring_neighbours",one_ring_neighbours)
        len_neighbors = len(one_ring_neighbours)
        area = get_barycentric_area(mesh,vertex)
        
        # calculate alpha and beta
        sum_of_cotangents = 0
        for j in range(len_neighbors):
            neighbor_index = one_ring_neighbours[j]
#             print("vertex",vertex)
#             print("neighbor_index",neighbor_index)
            alpha_beta = get_alpha_beta(mesh,vertex,neighbor_index)
#             print("alpha_beta",alpha_beta)
#             print("vertice, neigbour", [vertex,neighbor_index]) 
            cot_sum_vertex_difference = get_cot_sum_vertex_difference(mesh, vertex, alpha_beta, neighbor_index)
            sum_of_cotangents += cot_sum_vertex_difference 
        
#         print("sum_of_cotangents",sum_of_cotangents)
#         print("area", area)
        
        cotan_discretization[vertex,:] =  sum_of_cotangents/(2 * area)
        
    return cotan_discretization

In [26]:
del_f = get_cotan_discretization(M1)

In [80]:
H_cotan = get_mean_curvature(del_f, M1)

[[ 0.00365804  0.0636897  -0.23324129]
 [ 0.00139986 -0.06704217  0.21713489]
 [ 0.394377   -0.52606804  0.22558757]
 ...
 [ 0.01162711  0.05383023 -0.00257478]
 [ 0.13254474  0.00065782 -0.04827174]
 [ 0.26465269 -0.59357527 -0.45690909]]


## visualize mean curvature

In [81]:
colours_mean_curvature =  get_vertice_colours_from_list(H_cotan,cm.hot)

-0.6990100803196037
1.979070175532214


In [83]:
curvature_shading(M1,colours_mean_curvature)
m1_mesh_rd = pyrender.Mesh.from_trimesh(M1)
run_gui([m1_mesh_rd])

# curvature_shading(M1,colours_mean_curvature)
# m2_mesh_rd = pyrender.Mesh.from_trimesh(M2)
# run_gui([m2_mesh_rd])

## Q3 Spectral analysis

In [27]:
m1_fp = os.path.join('bumpy-cube-small.obj')
m2_fp = os.path.join(DEMO_PATH_CURVATURE,'lilium.obj')
m3_fp = os.path.join(DEMO_PATH_CURVATURE,'plane.obj')

assert os.path.exists(m1_fp), 'cannot found:'+m1_fp 
assert os.path.exists(m2_fp), 'cannot found:'+m2_fp 
assert os.path.exists(m3_fp), 'cannot found:'+m3-fp 

M1 = trimesh.load(m1_fp) 
M2 = trimesh.load(m2_fp) 
M3 = trimesh.load(m3_fp)

In [14]:
def get_mass_matrix(mesh):
    vertice_number = len(mesh.vertices)
    mass_matrix = np.zeros([vertice_number,vertice_number])
    mass_matrix_diag = np.zeros(vertice_number)
    for vertex in range(vertice_number):
        area = get_barycentric_area(mesh, vertex)
        mass_matrix[vertex,vertex] = area
        mass_matrix_diag[vertex] = area
    return [mass_matrix, mass_matrix_diag]

In [15]:
def get_c_value(mesh, vertex, neighbor_index, alpha_beta):
    c_value = 0
    if alpha_beta[1] == 0:
        c_value = cot(alpha_beta[0])/2
    else:
        c_value = (cot(alpha_beta[0]) + cot(alpha_beta[1]))/2
    if c_value == zoo:
        c_value = 0
    return c_value

In [16]:
def get_c_matrix(mesh):
    vertice_number = len(mesh.vertices)
    c_matrix = np.zeros([vertice_number,vertice_number])
    for vertex in range(vertice_number):
        one_ring_neighbours = mesh.vertex_neighbors[vertex]
        len_neighbors = len(one_ring_neighbours)
        
        sum_of_c_values = 0
        for j in range(len_neighbors):
            neighbor_index = one_ring_neighbours[j]
#             print("vertex",vertex)
#             print("neighbor_index",neighbor_index)
            alpha_beta = get_alpha_beta(mesh,vertex,neighbor_index)
            c_value = get_c_value(mesh, vertex, neighbor_index, alpha_beta)
            c_matrix[vertex, neighbor_index] = c_value
            sum_of_c_values = sum_of_c_values + c_value
        c_matrix[vertex, vertex] = -1 * sum_of_c_values
    return c_matrix    
        

In [17]:
def get_del_prime(M_diag, C):
    inv_sqrt_M = np.diag(np.power(M_diag, -0.5))
    del_prime = inv_sqrt_M @ C @ inv_sqrt_M
    return del_prime

In [18]:
def get_phi(M_diag, vecs):
    inv_sqrt_M = np.diag(np.power(M_diag, -0.5))
    phi = inv_sqrt_M @ vecs
    return phi

In [19]:
def get_spectral(channel, amount_of_eigen_vectors, phi_required, M):
    '''
    I am trying to do it in a matrix multiple kind of format, I first tile the channel
    to give a multiple n x amount_of_eigen_vectors, then I multiply wiht M and phi and take
    the diagonal to give me the dot products, then I use the diagonal to multiply the 
    amount_of phi I did, then sum them up. This would give me the spectral for one single
    channel, then repeat
    '''
    tiled_channel =  np.tile(channel,(amount_of_eigen_vectors,1))
    channel_dot_phi = np.diag(tiled_channel @ M @  phi_required) # remeber that you need to add M for the dot_product
#     print("channel_dot_phi",channel_dot_phi)
    solution = np.sum(channel_dot_phi * phi_required, axis=1)
#     print("solution",solution)
    return solution  

In [20]:
def get_spectral_mesh_xyz(mesh, amount_of_eigen_vectors, phi, M):
    phi_required = phi
    #x channel
    x_vertices = mesh.vertices[:,0]
    spectral_x = get_spectral(x_vertices, amount_of_eigen_vectors, phi_required, M)
    
    #y channel
    y_vertices = mesh.vertices[:,1]
    spectral_y = get_spectral(y_vertices, amount_of_eigen_vectors, phi_required, M)
    
    #z channel
    z_vertices = mesh.vertices[:,2]
    spectral_z = get_spectral(z_vertices, amount_of_eigen_vectors, phi_required, M)
    
    spectral_vertices = np.stack((spectral_x, spectral_y, spectral_z), axis=-1).real #stack lists by first transposing then
    return spectral_vertices

In [21]:
def get_cotan_colour_coding_v1(mesh, cmap):
    del_f = get_cotan_discretization(mesh)
    H = get_mean_curvature(del_f, mesh)
    colours_mean_curvature =  get_vertice_colours_from_list(H,cmap=cmap)
    return colours_mean_curvature

In [22]:
def show_mesh_with_curvature_colouring(mesh, curvature_colouring):
    curvature_shading(mesh,curvature_colouring)
    mesh_rd = pyrender.Mesh.from_trimesh(mesh)
    run_gui([mesh_rd])

In [23]:
def perform_spectral_analysis_and_display(mesh, eigen_vec_num):
    
    new_mesh = mesh.copy()
    
    M, M_diag = get_mass_matrix(new_mesh)
    
    C = get_c_matrix(new_mesh)
    
    del_prime = get_del_prime(M_diag, C)
    
    vals, vecs = eigsh(del_prime, k = eigen_vec_num, which='SM')
    
    phi = get_phi(M_diag, vecs)
    
    new_vert = get_spectral_mesh_xyz(new_mesh, eigen_vec_num, phi, M)
    
    new_mesh.vertices = new_vert
    
    curvature_colouring = get_cotan_colour_coding_v1(new_mesh, cm.hot)
    
    show_mesh_with_curvature_colouring(new_mesh, curvature_colouring)
    

In [45]:
eigen_vec_num = 30
perform_spectral_analysis_and_display(M1, eigen_vec_num)

[[ 0.00534434  0.26816652 -0.29490737]
 [-0.00063773 -0.25728067  0.27295833]
 [ 0.18037076 -0.23164651  0.28039376]
 ...
 [ 0.10011411  0.18461725 -0.05037302]
 [ 0.12448339  0.01359212 -0.11644513]
 [ 0.24501503 -0.20423188 -0.15774391]]
0.09719176088694612
4.1370186377649505


# More helper functions

In [24]:
def get_cotan_colour_coding(mesh, cmap):
    L = get_cotan_laplace_beltrami(mesh)
    del_x = L @ mesh.vertices
    H = get_mean_curvature(del_x, mesh)
    colours_mean_curvature =  get_vertice_colours_from_list(H,cmap=cmap)
    return colours_mean_curvature

In [25]:
def show_mesh_with_curvature_colouring(mesh, curvature_colouring):
    curvature_shading(mesh,curvature_colouring)
    mesh_rd = pyrender.Mesh.from_trimesh(mesh)
    run_gui([mesh_rd])

In [26]:
def get_single_vertex_neighbour_value(mesh, vertex, neighbor_index, alpha_beta, area):
    l_value = 0
    if alpha_beta[1] == 0:
        l_value = cot(alpha_beta[0])/(2*area)
    else:
        l_value = (cot(alpha_beta[0]) + cot(alpha_beta[1]))/(2*area)
        
    if l_value == zoo:
        l_value = 0
    return l_value

## Question 4

In [235]:
m1_fp = os.path.join('bumpy-cube-small.obj')
m2_fp = os.path.join(DEMO_PATH_SMOOTHING,'fandisk_ns.obj')
m3_fp = os.path.join(DEMO_PATH_SMOOTHING,'plane_ns.obj')


assert os.path.exists(m1_fp), 'cannot found:'+m1_fp 
assert os.path.exists(m2_fp), 'cannot found:'+m2_fp 
assert os.path.exists(m3_fp), 'cannot found:'+m3_fp 

M1 = trimesh.load(m1_fp) 
M2 = trimesh.load(m2_fp) 
M3 = trimesh.load(m3_fp)

### Get L

In [27]:
def get_cotan_laplace_beltrami(mesh):
    vertice_number = len(mesh.vertices)
    cotan_laplace_beltrami = np.zeros([vertice_number,vertice_number])
    for vertex in range(vertice_number):
        one_ring_neighbours = mesh.vertex_neighbors[vertex]
        len_neighbors = len(one_ring_neighbours)
        area = get_barycentric_area(mesh, vertex)
        sum_of_l_values = 0
        
        for j in range(len_neighbors):
            neighbor_index = one_ring_neighbours[j]
            alpha_beta = get_alpha_beta(mesh,vertex,neighbor_index)
            l_value = get_single_vertex_neighbour_value(mesh, vertex, neighbor_index, alpha_beta, area)
            cotan_laplace_beltrami[vertex, neighbor_index] = l_value
            sum_of_l_values = sum_of_l_values + l_value
        cotan_laplace_beltrami[vertex, vertex] = -1 * sum_of_l_values
    return cotan_laplace_beltrami

In [28]:
def explicit_smoothening(mesh, lamda):
    L = get_cotan_laplace_beltrami(mesh)
    vertice_number = len(mesh.vertices)
    I = np.eye(vertice_number)
    mesh.vertices = (I + (lamda * L)) @ mesh.vertices
    return mesh

In [29]:
def get_explicit_smoothened_mesh(mesh, iterations, lamda):
    new_mesh = mesh.copy() 
    for i in range(iterations):
        new_mesh =  explicit_smoothening(new_mesh, lamda) 
    return new_mesh

In [192]:
# M_explicit = get_explicit_smoothened_mesh(M3, 5, 2e-5)
M_explicit = get_explicit_smoothened_mesh(M3, 5, 0.11e-5)

In [194]:
curvature_colouring_explicit = get_cotan_colour_coding(M_explicit, cm.hot)
show_mesh_with_curvature_colouring(M_explicit, curvature_colouring_explicit)

[[ 2.22521806e+00  2.56170484e+00 -2.20261094e+01]
 [ 1.78734957e+01  8.49635123e+00  5.99144452e+01]
 [ 1.15340877e+01  2.10376991e-02 -2.81992981e+01]
 ...
 [ 5.64906828e+00 -3.92405852e+00 -5.46095169e+01]
 [ 1.95390300e+00 -9.36424551e+00  1.14673064e+01]
 [-4.79235156e-01 -1.27486688e+00 -4.80561121e+00]]
-12442.287924641823
9538.97469506864


## Plot original mesh

In [196]:
curvature_colouring_original_mesh = get_cotan_colour_coding(M3, cm.hot)
show_mesh_with_curvature_colouring(M3, curvature_colouring_original_mesh)

[[   6.48541033  -12.46058976   43.41634921]
 [  19.05586979   15.44393508   42.90097859]
 [  73.16587933   -7.79302687 -118.58270858]
 ...
 [   2.03590854  -14.70275498  -82.42829955]
 [   9.85035263  -33.84011045   24.82050437]
 [  -2.50624782   26.15476368   13.34807241]]
-392.2320444464382
487.7417383878694


# Question 5

In [210]:
m1_fp = os.path.join('bumpy-cube-small.obj')
m2_fp = os.path.join(DEMO_PATH_SMOOTHING,'fandisk_ns.obj')
m3_fp = os.path.join(DEMO_PATH_SMOOTHING,'plane_ns.obj')

assert os.path.exists(m1_fp), 'cannot found:'+m1_fp 
assert os.path.exists(m2_fp), 'cannot found:'+m2_fp 
assert os.path.exists(m3_fp), 'cannot found:'+m3_fp 

M1 = trimesh.load(m1_fp) 
M2 = trimesh.load(m2_fp) 
M3 = trimesh.load(m3_fp)

In [59]:
# L = get_cotan_laplace_beltrami(M5)

In [30]:
def get_conjugate_gradient_solution(A,B):
    X_k_new, info = cg(A, B)
    return X_k_new

In [31]:
def implicit_smoothening(mesh, L, lamda):
    vertice_number = len(mesh.vertices)
    X_k = mesh.vertices
    M, _ = get_mass_matrix(mesh)
    C = get_c_matrix(mesh)
    A = (M - lamda*C)
    B_x = (M @ X_k[:,0]) 
    B_y = (M @ X_k[:,1])
    B_z = (M @ X_k[:,2])
    
    #conjugate gradient x
    x_new = get_conjugate_gradient_solution(A, B_x).real
#     print("x_new",x_new)
    
    #conjugate gradient y
    y_new = get_conjugate_gradient_solution(A, B_y).real
#     print("y_new",y_new)
    
    #conjugate gradient z
    z_new = get_conjugate_gradient_solution(A, B_z).real
#     print("z_new",z_new)
    
    X_k_new = np.stack((x_new, y_new, z_new), axis=-1).real
    
    mesh.vertices = X_k_new
    return mesh

In [32]:
def get_implicit_smoothened_mesh(mesh, iterations, lamda):
    new_mesh = mesh.copy() 
    for i in range(iterations):
        L = get_cotan_laplace_beltrami(new_mesh)
        new_mesh =  implicit_smoothening(new_mesh, L, lamda) 
    return new_mesh

In [268]:
M_implicit = get_implicit_smoothened_mesh(M3, 5, 1e-4)

In [267]:
curvature_colouring_implicit = get_cotan_colour_coding(M_implicit, cm.hot)
show_mesh_with_curvature_colouring(M_implicit, curvature_colouring_implicit)

[[  0.50974918   0.57315276  -9.6287115 ]
 [  0.19191599   0.07378043  12.91602046]
 [ -0.76552732   0.75786642   6.21301688]
 ...
 [  1.10226567   0.6923687  -19.3184957 ]
 [ -0.64158275  -2.8850547   15.07264547]
 [  1.55435986   3.1589079   19.52663579]]
-17627689.022302743
910462578.9196613


In [58]:
del_6 = L6 @ M6.vertices

In [59]:
H6 = get_mean_curvature(del_6, M6)

[[ 0.00534841  0.11181609 -0.24628414]
 [ 0.00258212 -0.11918383  0.23739873]
 [ 0.18966686 -0.244912    0.17651895]
 ...
 [ 0.02153171  0.06632356 -0.00653425]
 [ 0.21949826  0.00524996 -0.1171625 ]
 [ 0.16753963 -0.24166471 -0.18807257]]


In [60]:
colours_mean_curvature_6 =  get_vertice_colours_from_list(H6,cm.jet)

-0.09789967419816611
2.0109883191327156


In [61]:
curvature_shading(M6,colours_mean_curvature_6)
m6_mesh_rd = pyrender.Mesh.from_trimesh(M6)
run_gui([m6_mesh_rd])

# Question 6

### Test laplacian mesh denoising

### Load meshes

In [38]:
m1_fp =  os.path.join(DEMO_PATH_CURVATURE,'plane.obj') 
m2_fp = os.path.join(DEMO_PATH_SMOOTHING,'fandisk_ns.obj')
m3_fp = os.path.join('bumpy-cube-small.obj')

assert os.path.exists(m1_fp), 'cannot found:'+m1_fp 
assert os.path.exists(m2_fp), 'cannot found:'+m2_fp 
assert os.path.exists(m3_fp), 'cannot found:'+m3_fp 

M1 = trimesh.load(m1_fp) 
M2 = trimesh.load(m2_fp) 
M3 = trimesh.load(m3_fp)

## access mesh properties

### add noise

In [33]:
def addNoise(mesh, noise_level, K = 1000, noise_mean = 0):
    '''
    INPUT: mesh - mesh to be changed
           noise_level - standard deviation of gaussian noise with 
           k - arbitary constant for scaling noise to length limith
        noise_mean - mean of the noise
    OUTPUT: mesh - mesh with noise
    '''
    new_mesh = mesh.copy()
    bounding_box_dimensions = new_mesh.extents
    noise = np.random.normal(noise_mean, noise_level, new_mesh.vertices.shape)
    boundedNoise = noise * (bounding_box_dimensions/K)
    new_mesh.vertices = new_mesh.vertices + boundedNoise
    return new_mesh

In [34]:
def get_implicit_smoothened_mesh(mesh, iterations, lamda):
    new_mesh = mesh.copy() 
    for i in range(iterations):
        L = get_cotan_laplace_beltrami(new_mesh)
        new_mesh =  implicit_smoothening(new_mesh, L, lamda) 
    return new_mesh

### Question 6 Solutiion

##  low level noise

In [45]:
noise_levels = [2]


for index, noise_level in enumerate(noise_levels):
#     print(noise_level)
    
    #add noise
    noisy_M3 = addNoise(M3, noise_level)
    
#     #show noisy mesh
#     curvature_colouring_noisy_M3 = get_cotan_colour_coding(noisy_M3, cm.hot)
#     show_mesh_with_curvature_colouring(noisy_M3, curvature_colouring_noisy_M3)
    
    #remove noise
    smoothened_noisy_M3 = get_implicit_smoothened_mesh(noisy_M3, 10, 1e-4)
    
    #show removed_noise_mesh
    curvature_colouring_smoothened_noisy_M3 = get_cotan_colour_coding(noisy_M3, cm.hot)
    show_mesh_with_curvature_colouring(smoothened_noisy_M3, curvature_colouring_smoothened_noisy_M3)

## medium level amount of noise

In [42]:
noise_levels = [10]

for index, noise_level in enumerate(noise_levels):
#     print(noise_level)
    
    #add noise
    noisy_M3 = addNoise(M3, noise_level)
    
    #show noisy mesh
    curvature_colouring_noisy_M3 = get_cotan_colour_coding(noisy_M3, cm.hot)
    show_mesh_with_curvature_colouring(noisy_M3, curvature_colouring_noisy_M3)
    
    #remove noise
    smoothened_noisy_M3 = get_implicit_smoothened_mesh(noisy_M3, 10, 5e-3)
    
    #show removed_noise_mesh
    curvature_colouring_smoothened_noisy_M3 = get_cotan_colour_coding(noisy_M3, cm.hot)
    show_mesh_with_curvature_colouring(smoothened_noisy_M3, curvature_colouring_smoothened_noisy_M3)

## extreme amount of noise

In [43]:
noise_levels = [40]

for index, noise_level in enumerate(noise_levels):
#     print(noise_level)
    
    #add noise
    noisy_M3 = addNoise(M3, noise_level)
    
    #show noisy mesh
    curvature_colouring_noisy_M3 = get_cotan_colour_coding(noisy_M3, cm.hot)
    show_mesh_with_curvature_colouring(noisy_M3, curvature_colouring_noisy_M3)
    
    #remove noise
    smoothened_noisy_M3 = get_implicit_smoothened_mesh(noisy_M3, 20, 1e-3)
    
    #show removed_noise_mesh
    curvature_colouring_smoothened_noisy_M3 = get_cotan_colour_coding(noisy_M3, cm.hot)
    show_mesh_with_curvature_colouring(smoothened_noisy_M3, curvature_colouring_smoothened_noisy_M3)